# [문제]
- law_2.docx 파일을 읽고, Chroma 저장
- LLM 질문 -> 답변
- 전세사기피해에 관한 법률 질문만 받기
- 이 외의 질문은 '답변을 할 수 없습니다.'

In [ ]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


## 1. 문서 내용 읽고 분할
loader = Docx2txtLoader('law_2.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
    length_function = len,
    is_separator_regex= False,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 벡터 데이터베이스에 저장
## 2.1 환경변수 읽어오기
load_dotenv()

## 2.2 임베딩 모델 지정
embedding = OpenAIEmbeddings(model = 'text-embedding-3-large')

## 2.3 벡터 데이터베이스에 저장
## [방법 1] in memory
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
# )


In [7]:

## [방법 2] Pinecone server
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

database = PineconeVectorStore.from_documents(
    embedding=embedding,
    index_name = 'law-3-index',
    documents= document_list,
)


In [6]:

## 3. 질문 -> 벡터 데이터베이스(vector store)에서 유사도 검색
## 3.1 사용자 질문
query = '전세사기 피해자 임대인의 국세계산은 어떻게 돼?'
# query = '오늘 점심 메뉴 정해줘~'

## 3.2 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=2)

## 3.3 문서 객체 -> 하나의 문자열
context = '😎😎\n'.join([doc.page_content for doc in retrieved_docs])

## 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달
## 4.1 프롬프트 작성
prompt = ''' 
[identity]
당신은 전세사기 피해 법률 전문가입니다.
아래는 전세사기 피해자 지원에 관한 법령 내용입니다.

- [context]를 참고하여 사용자의 질문에 답변하세요.
- 전세사기피해 법률 이외의 질문에는 '답변을 할 수 없습니다.'로 답하세요.

[context]
{retrieved_docs}

question: {query}
'''
## 4.2 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(
    retrieved_docs = context, 
    query=query)

## 4.3  LLM 모델 생성(hatOpenAI인스턴스 생성)
llm = ChatOpenAI(model= 'gpt-4o')

## 4.4 LLM 모델에 질문과 검색된 문서 보내기
ai_message = llm.invoke(formatted_prompt)
print(ai_message.content)

전세사기 피해자의 임대인에 대한 국세의 계산은 시행령 제3조에 따라 다음과 같이 이루어집니다. 임대인의 국세는 상속세, 증여세, 종합부동산세일 경우 각각의 고지 또는 신고 건별로 계산된 금액 중 더 큰 금액으로 산정됩니다. 구체적인 계산은 다음과 같습니다:

1. 상속세, 증여세 및 종합부동산세의 경우: 
   - 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 
   - 임대인이 체납한 고지 또는 신고 건별 국세 금액 중 과세 대상 재산에 부과된 국세 금액 중 큰 금액으로 결정됩니다.

2. 상속세, 증여세 및 종합부동산세 외의 국세의 경우: 
   - 각 고지 또는 신고 건별로 가목에 따라 계산한 금액으로 산정됩니다.

계산식에서 "전세사기 피해주택의 임대인이 보유한 모든 주택의 가격 합계액"은 임대인의 모든 주택의 시가표준액을 합산한 금액으로 하고, "전세사기 피해주택의 가격"은 해당 과세기준일의 시가표준액으로 하여 계산합니다.
